In [2]:
#!pip install gradio

In [3]:
import gradio as gr

In [4]:
import torch

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [5]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print(current_directory)


/upb/users/b/balram/profiles/unix/cs


In [6]:
from transformers import pipeline

2024-09-10 19:12:13.459606: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-10 19:12:13.484772: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-10 19:12:13.492542: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-10 19:12:13.511144: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-10 19:12:14.654550: W tensorflow/compiler/tf2

In [7]:
import re

In [8]:
import bitsandbytes as bnb

In [9]:
model_id = "meta-llama/Meta-Llama-3-8B"

In [10]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [11]:
# Load the model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,  # Enable 4-bit quantization
    device_map="auto",  # Automatically place model layers on GPU
    torch_dtype=torch.float16
    #quantization_config=bnb.BnbQuantizationConfig(load_in_4bit=True)
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
# Initialize the text-generation pipeline
text_generator = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer
    #device=0  # Use GPU 0
)

In [13]:
# Check if the model is on GPU or CPU
for param in model.parameters():
    print(f"Model is on: {param.device}")
    break  # Check one parameter

Model is on: cuda:0


In [14]:
# Assuming you have this function for generating text with a model
def generate_text(prompt_text,model_name, max_length=1500, num_return_sequences=1):
    if model_name == "LLama 3":
    # Your pre-defined text generation logic with LLama models
        response = text_generator(prompt_text, max_length=max_length, num_return_sequences=num_return_sequences)
    else:
        print("No model execution provided")
    return response  # Assuming response is a list and you want the first result

In [15]:
def extract_test_output(response):
    if response and len(response) > 0:
        generated_text = response[0].get('generated_text', '')
        match = re.search(r'Test Output:\s*(.*?)\s*(?:\n|$)', generated_text, re.DOTALL)
        if match:
            return match.group(1).strip()
    return 'Output not found'

In [16]:
def parse_model_output(model_output):
    # Initialize an empty list to store triples
    triples = []
    
    # Remove leading/trailing whitespace and split the input data into lines
    lines = [line.strip() for line in model_output.strip().split('\n')]
    
    # Define the regex pattern to match triples in the format: relation(subject, object)
    pattern = re.compile(r'(.+?)\s*\(([^,]+),\s*([^)]+)\)')
    
    for line in lines:
        # Find all matches for the pattern in the line
        matches = pattern.findall(line)
        
        for match in matches:
            relation, subject, obj = match
            # Clean up subject and object values
            subject = subject.strip()
            obj = obj.strip()
            
            # Append the extracted triple to the list
            triples.append({"sub": subject, "rel": relation, "obj": obj})
    
    # Return the list of triples
    return triples

In [17]:
# Function to process the file and generate RDF triples
def process_file(file_path, model_name):
    # Read the uploaded file
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Prompt Construction
    prompt = f"""
    Here is the RDF data:

    {text}
    """

    print(f"Prompt:\n{prompt}")
    
    # Generate RDF triples using the selected model
    model_response = generate_text(prompt,model_name, max_length=500)
   # response = generate_text(prompt_text)
    test_output = extract_test_output(model_response)
    
    # Debugging: Print the test output to ensure it's correctly extracted
    print("test_output : ",test_output)
    # Parse the test output into triples
    triples = parse_model_output(test_output)      
    # Debugging: Print the parsed triples to ensure they're correct
    print("parsed triples : ",triples)
    # Returning both the file content and RDF triples for display
    #rdf_triples = parse_model_output(response)
   # print("rdf_triples",rdf_triples)
    output = f"\nGenerated RDF Triples:\n{triples}"
    
    return output

In [18]:
# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# RDF Triple Generator")
    gr.Markdown("Upload a text file, select a model, and generate RDF triples using LLama models.")

    # File upload component with filepath type
    file_input = gr.File(label="Upload Input Text File", type="filepath")
    
    # Dropdown menu for model selection
    
    selected_model = gr.Dropdown(choices=["LLama 3", "LLama 2"], label="Select Model")
    
    # Generate button to trigger RDF generation
    generate_button = gr.Button("Generate RDF Triples")
    
    # Textbox to display the generated RDF triples
    rdf_output = gr.Textbox(label="Generated RDF Triples", lines=100)

    # Connect the button to the processing function
    generate_button.click(fn=process_file, inputs=[file_input, selected_model], outputs=rdf_output)

In [19]:
# Launch the Gradio interface
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://47b6639efc442734d9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt:

    Here is the RDF data:

    prompt_text:
Given the following ontology and sentences, please extract the triples from the sentence according to the relations in the ontology. In the output, only include the triples in the given output format.
CONTEXT:
Ontology Concepts: book, literary work, author, publisher, International Standard Book Number, library, calender date, film, fictional character, writer, scientific journal, article, human, literary genre, publication, trade magazine, language, intellectual work, country, territory,
Ontology Relations: illustrator(,human), followed_by(,), publication_date(book,), author(book,human), publisher(book,publisher), characters(literary work,fictional character), editor(trade magazine,human), place_of_publication(,), narrative_location(literary work,territory), genre(literary work,literary genre), language_of_work_or_name(literary work,language), depicts(,)

Example Sentence: The Foundling and Other Tales of Prydain is a collection of 

/upb/users/b/balram/profiles/unix/cs/.local/lib/python3.12/site-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


test_output :  illustrator(Peter and the Piskies: Cornish Folk and Fairy Tales,Raymond Briggs)
parsed triples :  [{'sub': 'Peter and the Piskies: Cornish Folk and Fairy Tales', 'rel': 'illustrator', 'obj': 'Raymond Briggs'}]


In [21]:
#Please ignore below celss

In [25]:
response = [{'generated_text': '\n    Here is the RDF data:\n\n    prompt_text:\nGiven the following ontology and sentences, please extract the triples from the sentence according to the relations in the ontology. In the output, only include the triples in the given output format.\nCONTEXT:\nOntology Concepts: book, literary work, author, publisher, International Standard Book Number, library, calender date, film, fictional character, writer, scientific journal, article, human, literary genre, publication, trade magazine, language, intellectual work, country, territory,\nOntology Relations: illustrator(,human), followed_by(,), publication_date(book,), author(book,human), publisher(book,publisher), characters(literary work,fictional character), editor(trade magazine,human), place_of_publication(,), narrative_location(literary work,territory), genre(literary work,literary genre), language_of_work_or_name(literary work,language), depicts(,)\n\nExample Sentence: The Foundling and Other Tales of Prydain is a collection of short high fantasy stories for children by Lloyd Alexander and illustrator Margot Zemach.\nExample Output: illustrator(The Foundling and Other Tales of Prydain,Margot Zemach)\n\nTest Sentence: Peter and the Piskies: Cornish Folk and Fairy Tales is a 1958 anthology of 34 fairy tales from Cornwall that have been collected and retold by Ruth Manning-Sanders and illustrated by Raymond Briggs.\nTest Output:\n     illustrator(Peter and the Piskies: Cornish Folk and Fairy Tales,Raymond Briggs)\n     publication_date(Peter and the Piskies: Cornish Folk and Fairy Tales,1958)\n     author(Peter and the Piskies: Cornish Folk and Fairy Tales,Ruth Manning-Sanders)\n     publisher(Peter and the Piskies: Cornish Folk and Fairy Tales,McMillan Publishing)\n     characters(Peter and the Piskies: Cornish Folk and Fairy Tales,Peter)\n     characters(Peter and the Piskies: Cornish Folk and Fairy Tales,Piskies)\n     language_of_work_or_name(Peter and the Piskies: Cornish Folk and Fairy Tales,Cornish)\n     language_of_work_or_name(Peter and the Piskies: Cornish Folk and Fairy Tales,English)\n     narrative_location(Peter and the Piskies: Cornish Folk and Fairy Tales,Cornwall)\n     genre(Peter and the Piskies: Cornish Folk and'}]

In [26]:
response

[{'generated_text': '\n    Here is the RDF data:\n\n    prompt_text:\nGiven the following ontology and sentences, please extract the triples from the sentence according to the relations in the ontology. In the output, only include the triples in the given output format.\nCONTEXT:\nOntology Concepts: book, literary work, author, publisher, International Standard Book Number, library, calender date, film, fictional character, writer, scientific journal, article, human, literary genre, publication, trade magazine, language, intellectual work, country, territory,\nOntology Relations: illustrator(,human), followed_by(,), publication_date(book,), author(book,human), publisher(book,publisher), characters(literary work,fictional character), editor(trade magazine,human), place_of_publication(,), narrative_location(literary work,territory), genre(literary work,literary genre), language_of_work_or_name(literary work,language), depicts(,)\n\nExample Sentence: The Foundling and Other Tales of Pryda

In [16]:
 # Prompt Construction
prompt = f"""
    Here is the RDF data:
    Barack Hussein Obama II[a] (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party, he was the first African-American president in U.S. history. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004.
    Obama was born in Honolulu, Hawaii. He graduated from Columbia University in 1983 with a Bachelor of Arts degree in political science and later worked as a community organizer in Chicago. In 1988, Obama enrolled in Harvard Law School, where he was the first black president of the Harvard Law Review. He became a civil rights attorney and an academic, teaching constitutional law at the University of Chicago Law School from 1992 to 2004. He also went into elective politics; Obama represented the 13th district in the Illinois Senate from 1997 until 2004, when he successfully ran for the U.S. Senate. In the 2008 presidential election, after a close primary campaign against Hillary Clinton, he was nominated by the Democratic Party for president. Obama selected Joe Biden as his running mate and defeated Republican nominee John McCain.
    

    Please parse and extract RDF-like triplets from the text. For each identified triplet, provide the output as a list of tuples.

    Each tuple should follow the structure:
    ("Subject", "Predicate", "Object")

    Guidelines:
    1. Ensure that each triplet has exactly three components: a subject, a predicate, and an object.
    2. If the text mentions a fact but lacks one of these components, reformulate it or omit it to ensure the triplet is complete.
    3. Handle various types of information, including names, locations, dates, and numerical values, in a similar format.

    Examples:
    Text: "Obama was born in Honolulu, Hawaii. He graduated from Columbia University in 1983."

    Example Output:
    extracted_triplets = [
        ("Barack Hussein Obama II", "was born in", "Honolulu, Hawaii"),
        ("Barack Hussein Obama II", "graduated from", "Columbia University"),
        ("Barack Hussein Obama II", "graduated in", "1983"),
    ]

   Example Text: "Obama was the first African-American president in U.S. history."

    Example Output:
    extracted_triplets = [
        ("Barack Hussein Obama II", "was the first African-American president in", "U.S. history"),
    ]

    Ensure that each extracted triplet is fully formed and logically coherent.
    """


In [ ]:
# Function to process the file and generate RDF triples
def process_file(file_path, model_name):
    # Read the uploaded file
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Prompt Construction
    prompt = f"""
    Here is the RDF data:

    {text}

    Please parse and extract RDF-like triplets from the text. For each identified triplet, provide the output as a list of tuples.

    Each tuple should follow the structure:
    ("Subject", "Predicate", "Object")

    Guidelines:
    1. Ensure that each triplet has exactly three components: a subject, a predicate, and an object.
    2. If the text mentions a fact but lacks one of these components, reformulate it or omit it to ensure the triplet is complete.
    3. Handle various types of information, including names, locations, dates, and numerical values, in a similar format.

    Examples:
    Text: "Obama was born in Honolulu, Hawaii. He graduated from Columbia University in 1983."

    Output:
    extracted_triplets = [
        ("Barack Hussein Obama II", "was born in", "Honolulu, Hawaii"),
        ("Barack Hussein Obama II", "graduated from", "Columbia University"),
        ("Barack Hussein Obama II", "graduated in", "1983"),
    ]

    Text: "Obama was the first African-American president in U.S. history."

    Output:
    extracted_triplets = [
        ("Barack Hussein Obama II", "was the first African-American president in", "U.S. history"),
    ]

    Ensure that each extracted triplet is fully formed and logically coherent.
    """

    print(f"Prompt:\n{prompt}")

    
    # Generate RDF triples using the selected model
    response = generate_text(prompt,model_name, max_length=1500)
        # Returning both the file content and RDF triples for display
    rdf_triples = extract_triplets(response)
    output = f"File Content:\n{text}\n\nGenerated RDF Triples:\n{rdf_triples}"
    
    return output